In [ ]:
import wikipedia
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration # Tokenizer và Model T5 (dùng cho tóm tắt)
import textwrap # textwrap để format văn bản
import warnings # warnings để kiểm soát cảnh báo
import requests

# Tắt cảnh báo không quan trọng từ wikipedia
warnings.filterwarnings("ignore", category=UserWarning, module='wikipedia')
print("Các thư viện đã được import")

d:\CNLTHD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Các thư viện đã được import


In [2]:
# --- Cấu hình ---
MODEL_NAME = "t5-small"  # Hoặc "t5-base", "t5-large", "google/mt5-small" (cho nhiều ngôn ngữ)
WIKI_PAGE_TITLE = "Artificial intelligence" # Thay đổi thành tiêu đề trang Wikipedia bạn muốn tóm tắt
WIKI_LANGUAGE = 'en'      # Đặt ngôn ngữ ('vi' cho tiếng Việt, 'en' cho tiếng Anh,...)
MAX_OUTPUT_LENGTH = 300   # Độ dài tối đa của bản tóm tắt (số lượng token)
MIN_OUTPUT_LENGTH = 50    # Độ dài tối thiểu của bản tóm tắt
NUM_BEAMS = 5             # Sử dụng beam search để tạo kết quả tốt hơn (thường 4-5)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Sử dụng GPU nếu có

print(f"Cấu hình:")
print(f"  Model: {MODEL_NAME}")
print(f"  Trang Wikipedia: '{WIKI_PAGE_TITLE}' ({WIKI_LANGUAGE})")
print(f"  Độ dài tóm tắt: [{MIN_OUTPUT_LENGTH}, {MAX_OUTPUT_LENGTH}] tokens")
print(f"  Thiết bị: {DEVICE}")

Cấu hình:
  Model: t5-small
  Trang Wikipedia: 'Artificial intelligence' (en)
  Độ dài tóm tắt: [50, 300] tokens
  Thiết bị: cpu


In [3]:
local_model = "./t5_small_local"

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval() # Chuyển sang chế độ đánh giá
print("Model và Tokenizer đã tải xong và sẵn sàng trên thiết bị:", DEVICE)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Model và Tokenizer đã tải xong và sẵn sàng trên thiết bị: cpu


In [5]:
def get_wikipedia_content(page_title, language='vi'):
    """Lấy nội dung tóm tắt (summary) hoặc toàn bộ nội dung từ trang Wikipedia."""
    content = None
    page_obj = None
    try:
        wikipedia.set_lang(language) # Đặt ngôn ngữ
        # Thử lấy trang, cho phép gợi ý và chuyển hướng
        page_obj = wikipedia.page(page_title, auto_suggest=True, redirect=True)

        # Lấy toàn bộ nội dung
        content = page_obj.content
        print(f"Đã tìm thấy và lấy nội dung trang: '{page_obj.title}' (URL: {page_obj.url})")
        # Kiểm tra xem nội dung có rỗng không
        if not content or content.isspace():
             print(f"Cảnh báo: Nội dung trang '{page_obj.title}' có vẻ rỗng.")
             content = None # Đặt lại thành None nếu rỗng

    except wikipedia.exceptions.PageError:
        print(f"LỖI: Không tìm thấy trang Wikipedia với tiêu đề khớp chính xác hoặc gần đúng với '{page_title}'.")
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"LỖI: Tiêu đề '{page_title}' không rõ ràng (Disambiguation).")
        print("Các trang có thể liên quan (tối đa 5):")
        for i, option in enumerate(e.options[:5]):
             print(f"  {i+1}. {option}")
        print("Vui lòng thử lại với một trong các tiêu đề cụ thể hơn ở trên.")
    except requests.exceptions.RequestException as e:
         print(f"LỖI MẠNG: Không thể kết nối tới Wikipedia. Chi tiết: {e}")
    except Exception as e:
        print(f"LỖI không xác định khi truy cập Wikipedia: {type(e).__name__} - {e}")

    return content, page_obj # Trả về cả nội dung và đối tượng trang

In [7]:
def summarize_text(text, model, tokenizer, max_out_len, min_out_len, num_beams, device, model_name="t5"):
    """Tóm tắt văn bản đầu vào sử dụng model T5 đã tải."""
    if not text or text.isspace():
        print("Cảnh báo: Không có nội dung hợp lệ để tóm tắt.")
        return 
       
    # Xác định tiền tố dựa trên model (đa số T5 dùng "summarize: ")
    # Một số model đa ngôn ngữ (như mT5) có thể không cần hoặc dùng tiền tố khác
    prefix = "summarize: "
    if "mt5" in model_name.lower():
        print("Phát hiện model mT5, không sử dụng tiền tố 'summarize: ' (hoặc thử nghiệm)")

    input_text = prefix + text

    # Tokenize văn bản đầu vào
    # Giới hạn độ dài đầu vào quan trọng cho T5 (512 hoặc 1024 là phổ biến)
    # Kiểm tra max_position_embeddings của model nếu không chắc
    try:
      input_max_length = tokenizer.model_max_length
      if input_max_length > 2048: # Giới hạn hợp lý phòng trường hợp model config lạ
          input_max_length = 1024
      print(f"Độ dài tối đa đầu vào cho tokenizer: {input_max_length}")
    except AttributeError:
      input_max_length = 1024 # Giá trị mặc định an toàn
      print(f"Không tìm thấy model_max_length, sử dụng mặc định: {input_max_length}")


    # Token hóa đầu vào
    input_ids = tokenizer.encode(input_text,
                                 return_tensors='pt',
                                 max_length=input_max_length,
                                 truncation=True,
                                 padding='longest').to(device)

    input_token_length = input_ids.shape[1]
    print(f"Độ dài token đầu vào thực tế (sau khi cắt nếu cần): {input_token_length}")
    if input_token_length == input_max_length:
        print("Đầu vào đã bị cắt bớt do dài hơn giới hạn của tokenizer.")

    print(f"Đang tạo tóm tắt (max_len={max_out_len}, min_len={min_out_len}, num_beams={num_beams})...")

    # Tạo tóm tắt sử dụng phương thức generate của model
    summary = "Lỗi trong quá trình tạo tóm tắt." # Giá trị mặc định nếu có lỗi
    try:
        with torch.no_grad(): # Không cần tính gradient khi inference
            summary_ids = model.generate(
                input_ids,
                max_length=max_out_len + 2, # Thêm chút không gian cho token đặc biệt có thể xuất hiện
                min_length=min_out_len,
                num_beams=num_beams,
                length_penalty=2.0,     # Khuyến khích câu dài hơn một chút (có thể điều chỉnh)
                early_stopping=True     # Dừng sớm khi hội tụ
            )
    
        # Giải mã (decode) các token output thành văn bản
        print("Đang giải mã kết quả...")
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    except Exception as e:
        print(f"\nLỖI trong quá trình model.generate() hoặc decode(): {type(e).__name__} - {e}")
    return summary

In [8]:
# Gọi hàm để lấy nội dung
wiki_content, wiki_page = get_wikipedia_content(WIKI_PAGE_TITLE, language=WIKI_LANGUAGE)

# Kiểm tra xem nội dung có lấy được không
if wiki_content:
    print(f"\nĐã lấy được nội dung cho '{wiki_page.title}'.")
    # In ra một vài dòng đầu tiên của nội dung gốc
    print("\n--- 500 ký tự đầu của nội dung gốc ---")
    print(textwrap.fill(wiki_content[:500] + "...", width=100))
else:
    print(f"\nKhông lấy được nội dung cho '{WIKI_PAGE_TITLE}'. Không thể tiếp tục tóm tắt.")

Đã tìm thấy và lấy nội dung trang: 'Artificial intelligence' (URL: https://en.wikipedia.org/wiki/Artificial_intelligence)

Đã lấy được nội dung cho 'Artificial intelligence'.

--- 500 ký tự đầu của nội dung gốc ---
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically
associated with human intelligence, such as learning, reasoning, problem-solving, perception, and
decision-making. It is a field of research in computer science that develops and studies methods and
software that enable machines to perceive their environment and use learning and intelligence to
take actions that maximize their chances of achieving defined goals. High-profile applications of AI
incl...


In [9]:
# Chỉ chạy tóm tắt nếu có nội dung
generated_summary = None
if wiki_content:
    # Thực hiện tóm tắt
    generated_summary = summarize_text(
        wiki_content,
        model,
        tokenizer,
        MAX_OUTPUT_LENGTH,
        MIN_OUTPUT_LENGTH,
        NUM_BEAMS,
        DEVICE,
        model_name=MODEL_NAME # Truyền tên model để xử lý prefix nếu cần
    )
    # In kết quả tóm tắt
    print("\n" + "="*40 + " BẢN TÓM TẮT " + "="*40)
    if generated_summary:
        print(textwrap.fill(generated_summary, width=100))
    else:
        print("Không tạo được bản tóm tắt.")
    print("="*93)

else:
    print("\nKhông có nội dung từ Wikipedia.")

Độ dài tối đa đầu vào cho tokenizer: 512
Độ dài token đầu vào thực tế (sau khi cắt nếu cần): 512
Đầu vào đã bị cắt bớt do dài hơn giới hạn của tokenizer.
Đang tạo tóm tắt (max_len=300, min_len=50, num_beams=5)...
Đang giải mã kết quả...

======================================== BẢN TÓM TẮT ========================================
AI is a field of research in computer science that develops and studies methods and software that
enable machines to perceive their environment. high-profile applications include advanced web search
engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix);
virtual assistants (e.g., Google Assistant, Siri, and Alexa); generative and creative tools (e.g.,
language models and AI art)
